In [1]:
import json
import os
from pathlib import Path
import librosa
import numpy as np
from typing import Dict, List
from tqdm import tqdm

In [2]:
FILE_NAME = "data_TEST"
CLASSES = ["RMT", "HRK"]

SAMPLE_RATE = 10000
SEGMENT_DURATION = 0.0256  # duration of each segment in seconds
OVERLAP_DURATION = 0   # overlap duration in % (0 means no overlap)
NUM_MFCC = 16
N_FFT = 2 ** int(np.ceil(np.log2(SEGMENT_DURATION * SAMPLE_RATE)))
SEGMENT_HOP_LENGTH = int(SAMPLE_RATE * SEGMENT_DURATION * (1 - OVERLAP_DURATION / 100))
MFCC_HOP_LENGTH = SEGMENT_HOP_LENGTH

In [3]:
DATASET_PATH = "./data/original_dataset"
JSON_PATH = "./data/" + FILE_NAME + ".json"

In [4]:
print(f"n_fft: {N_FFT}, hop_length: {SEGMENT_HOP_LENGTH}")

n_fft: 256, hop_length: 256


In [5]:
def extract_mfcc(file_path: str, num_mfcc: int, n_fft: int, hop_length: int) -> List[List[float]]:
    """Extract MFCCs from an audio file."""
    signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)
    track_duration = librosa.get_duration(y=signal, sr=SAMPLE_RATE)
    samples_per_track = SAMPLE_RATE * track_duration
    samples_per_segment = int(SAMPLE_RATE * SEGMENT_DURATION)
    num_segments = int((samples_per_track - samples_per_segment) / hop_length) + 1

    mfccs = []
    for d in range(num_segments):
        start = hop_length * d
        finish = start + samples_per_segment
        if finish > len(signal):
            finish = len(signal)
        mfcc = librosa.feature.mfcc(y=signal[start:finish], sr=SAMPLE_RATE, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length, center=False)
        mfccs.append(mfcc.T.tolist())
    return mfccs

In [6]:
def save_mfcc(dataset_path: str, json_path: str, selected_classes: List[str], voc_only: bool = True, num_mfcc: int = 16, n_fft: int = 256, hop_length: int = 256) -> None:
    """Extracts MFCCs from audio dataset and saves them into a json file along with class labels."""
    data: Dict[str, List] = {"mapping": [], "patients": {}}
    dataset_path = Path(dataset_path)

    # Create a dictionary to map the selected classes to their respective labels
    class_to_label = {c: i for i, c in enumerate(selected_classes)}

    # Walk through dataset path
    for dirpath, _, filenames in os.walk(dataset_path):
        if dirpath != str(dataset_path):
            semantic_label = Path(dirpath).name
            if semantic_label in ["DPR", "HRK"]:
                if "DPR+HRK" in selected_classes:
                    semantic_label = "DPR+HRK"
                elif semantic_label not in selected_classes:
                    continue
            
            if semantic_label in selected_classes:
                if semantic_label not in data["mapping"]:
                    data["mapping"].append(semantic_label)
                print(f"\nProcessing: {semantic_label}")

                for f in tqdm(filenames, desc=f"Processing files in {semantic_label}", leave=False):
                    if ('voc' in f) == voc_only:
                        file_path = str(Path(dirpath) / f)
                        mfccs = extract_mfcc(file_path, num_mfcc, n_fft, hop_length)
                        patient_data = {
                            "mfcc": mfccs,
                            "label": class_to_label[semantic_label],
                            "file": file_path
                        }
                        data["patients"][file_path] = patient_data
                        print(f"{file_path}, segments: {len(mfccs)}")

    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)
    
    print("\nDONE")

In [7]:
save_mfcc(DATASET_PATH, JSON_PATH, selected_classes=CLASSES, voc_only=True, num_mfcc=NUM_MFCC, n_fft=N_FFT, hop_length=SEGMENT_HOP_LENGTH)


Processing: HRK


Processing files in HRK:   0%|          | 0/20 [00:00<?, ?it/s]

Processing files in HRK:  15%|█▌        | 3/20 [00:53<05:02, 17.82s/it]

data/original_dataset/HRK/020806nt1_intvoc.wav, segments: 7080


Processing files in HRK:  25%|██▌       | 5/20 [04:06<14:01, 56.11s/it]

...

data/original_dataset/RMT/100903hnf_intvoc.wav, segments: 1716


Processing files in RMT:  85%|████████▌ | 17/20 [09:16<01:41, 33.80s/it]

data/original_dataset/RMT/021406nt2_intvoc.wav, segments: 10901


data/original_dataset/RMT/011706nt1_intvoc.wav, segments: 8717



DONE
